#### Sensitivity Analysis 
Simulation of the effect of Energy Producer Maximum outPut on the Voltage Rise Detected 

Simulation based on  : 
* Prediction block RNN trained in [RNN_Train_StLaurentDeJourdes](RNN_Train_StLaurentDeJourdes.ipynb) 
    * To predict $X(k)$ and $Y(k)$  as respectively  $\tilde{X(k)}$ an  $\tilde{Y(k)}$ (Predictions on testing set are done in [RNN_Sim_StLaurentDeJourdes](RNN_Sim_StLaurentDeJourdes.ipynb))
* Robust Combined RNN To predict $X(k)$ and $Y(k)$  as respectively  $\tilde{X(k)}$ an  $\tilde{Y(k)}$ based on robust Model1 and Model 3 [2021_2022_RNN_Robust_All_Models](2021_2022_RNN_Robust_All_Models.ipynb))
* Prediction Block Future Known [2021_2022_KnownFuture](2021_2022_KnownFuture.ipynb)
* Persistence Model [2021_2022_Persistence](2021_2022_Persistence.ipynb)


Maximum voltage rise $vm_{pu}^{max} = 1.0250 $

---

#### Define the parallel block PF/OPF

In [1]:
def par_block_pfOpf(par_engines,
                    pred_model_f=None
                   ):
    """ Block PF/OPF using parallels engines.

    Execute a power flow, an optimal power flow, or both depending on ``opf_status``
    extracted from the parallel engine object ``par_engines``. If ``opf_status`` is
    "Both", the function is used as the block PF/OPF. If ``opf_status`` is  ``False``,
    the function is used as the block PF..

    Parameters
    ----------
    par_engines:  ipyparallel.cluster
        Parallel engines object,, Instance of  :py:class:`oriClass.CreateParEngines`.
    pred_model_f: str, Optional, Default =  None
        'Pers' ==> Using persistence model.

    Returns
    -------
    pandas.DataFrame
        Output of the function :py:func:`oriClass.CreateParEngines.get_results_asDf`.
        
    Warnings
    ---------
    DO NOT CALL this function from the module :py:mod:`oriFunctions`. This function
    **Must** be implemented in the local space of each notebook that use it as done in 
    `VoltageRiseBinaryUpdated<https://github.com/pajjaecat/ORI-SRD/blob/main/Ressources/Notebooks/VoltageRiseBinaryUpdated.ipynb>`_
    for instance.

    """

    # Note that all the variables used in the parallel running MUST be  already sent to the local space of each engine
    if pred_model_f == 'Pers':
        # Run problem in parallel
        %px par_run_Results = [par_oriFunctions.run_powerflow_at(lowerNet, cur_period+1, lowerNet_hv_activated_bus, sum_max_main_network,  dict_df_sgenLoad, vm_mu_max, opf_status, pred_model) for cur_period in period_part]
    else:
        %px par_run_Results = [par_oriFunctions.run_powerflow_at(lowerNet, cur_period, lowerNet_hv_activated_bus, sum_max_main_network,  dict_df_sgenLoad, vm_mu_max, opf_status) for cur_period in period_part]

    # Gather the results of all the engines in a unique variable.
    results = par_engines.gather_results('par_run_Results')
    
    # Wait 2seconds time for gathering the results of parralel computing.
    # This waiting time could be reduce when using more powerful machines.
    time.sleep(2)

    # Extract results
    extracted_results = par_engines.get_results_asDf() 

    # Return Extacted results 
    return extracted_results

#### Import Modules 


In [2]:
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # Profiling 
import seaborn as sbn
import pickle, sys, importlib,  time
import os
from pickle import load
import joblib

In [3]:
# import psutil
function_folder = '../Modules/' 
# Add function folder to path if it is not already
if function_folder not in sys.path: sys.path.append(function_folder)

import oriFunctions as oriFc
import checker
import oriClass as oriCs
from oriVariables import (network_folder,
                          defAuth_hvBus_vRiseMax,
                          default_hv_voltage,
                          default_lv_voltage,
                          defAuth_hvBus_vRiseMin, 
                          excel_folder, 
                          h_start_end,
                          train_split_date,  
                          testSet_end_date,
                          testSet_start_date, 
                          testSet_start_date_M2, 
                          default_ctrld_hvProd_max,
                          Δt
                          )

#### Import Cleaned data files for component of the network

In [4]:
df_data = joblib.load(network_folder+'CleanedData.pkl')

# create daylight periods
per_index = df_data.index
per_daylight = ( pd.Series(index=per_index.to_timestamp(), dtype=object)
                .between_time(*h_start_end) ).index.to_period('10T')

# From the daylight periods extract for the first element the firsts year, month and day.
starting_year = per_daylight[0].year 
starting_month = per_daylight[0].month 
starting_day = per_daylight[0].day 

# Extract the total number of period during daylight time of a day 
day_tot_per = len(per_daylight[(per_daylight.year == starting_year)
                               &(per_daylight.month == starting_month)
                               &(per_daylight.day == starting_day) ] )

# Extract daylight periods
df_data = df_data.loc[per_daylight]


# # Extract only the relavant testing set
df_final = df_data[(df_data.index>= testSet_start_date_M2) & (df_data.index<= testSet_end_date)]

# Create period index 1 and 2 
per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                        ).between_time('07:10','18:50') ).index.to_period('10T')

#### Define Controlled HV's name and extract Non controlled HV names

In [5]:
ctrld_HvProdName = 'P0100'  # Define the name of the controlled HV Sgen

# Extract all the HV producer and get the name of Non - controlled HV Prod
hvProdNames = {elm for elm in df_final.columns if elm[-4:].isdecimal()}
non_ctrld_hvProdNames_list = list(hvProdNames.difference({ctrld_HvProdName}))

#### Load prediction file of model1 
i.e. $\tilde{X(k)}$ an  $\tilde{Y(k)}$ predicted in [RNN_Sim_StLaurentDeJourdes](RNN_Sim_StLaurentDeJourdes.ipynb)

In [6]:
rnnPred_df = joblib.load(f'{network_folder}simulationResults/RNN_pred.pkl')

#### Start Parallel engines

In [7]:
n_engines = os.cpu_count()-1         # Total number of engines
parEngines = oriCs.CreateParEngines(n_engines)

Starting 7 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/7 [00:00<?, ?engine/s]

### Import Networks


In [8]:
lowerNet=pp.from_pickle(f'{network_folder}CIVAUX.p')
upperNet=pp.from_pickle(f'{network_folder}ST LAURENT.p')

# Init an object of InitNetworks
networks = oriCs.InitNetworks(upperNet, lowerNet)

### Operation on network parameters

In [9]:
# Defined Voltage rise threshold
vm_mu_max, vm_mu_min = defAuth_hvBus_vRiseMax, defAuth_hvBus_vRiseMin  

# Inititialize the controllable hv prod in the network
networks.init_controlled_hvProd(ctrld_HvProdName)

# Extract the HV bus from the list of activated bus 
lowerNet_hv_activated_bus = networks.get_lowerNet_hvActivatedBuses(lowerNet
                                                                   .bus
                                                                   .query('vn_kv==@default_hv_voltage').index)
# Add maximum voltage constraints on the Hv Buses in the network 
networks.lowerNet_set_vrise_threshold(lowerNet_hv_activated_bus, vm_mu_min, vm_mu_max)

### Controllable Generators costs

In [10]:
# get the index of the controlled HV prod in SGEN table
ctrld_hvProd_index = lowerNet.sgen.query('name==@ctrld_HvProdName').index

# Add negative cost to usability of sgen P0100 so its usage can be maximised while respecting the constraints on the network
pp.create_poly_cost(lowerNet, ctrld_hvProd_index,'sgen', cp1_eur_per_mw=-1)

lowerNet.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,23,sgen,0.0,-1.0,0.0,0.0,0.0,0.0


### Simulations

#### Define Data to use in local space of engines

In [11]:
pred_model = 'Pers'   # Define the prediction Model 

# Create a dictionnary containing all the variable that will be used by each client or engine
parameters_dict = dict({'lowerNet':lowerNet, 
                        'lowerNet_hv_activated_bus':lowerNet_hv_activated_bus, 
                        'sum_max_main_network':networks.get_upperNet_sum_max_lvProdLoad(), 
                        'vm_mu_max':vm_mu_max,
                        'pred_model':pred_model
                         })

---

#### Robust Persistence

**<span style='color:Orange'>  The results of the Robust Persistence simulation using the default networks (Civaux and St Laurent) is availaible in</span>** [SensAnalysisP0100_PersRob](../Pickle_files/simulationResults/SensAnalysisP0100_PersRob.pkl). 

##### Define Range of variation for the maximum power output of the controlled HV Prod i.e. P0100 

In [12]:
ctrldHvProd_output_var_step = 0.2   # (MW) Controlled HV Prod output variation step 
ctrld_hvProd_max_range = np.arange(0, default_ctrld_hvProd_max+0.1 ,ctrldHvProd_output_var_step)

##### For each element in ctrld_hvProd_max_range, run a simulation and save the results 

In [ ]:
sim_res_Dict = {}  # Dictionnary to save variables

for cur_index in range(len(ctrld_hvProd_max_range)):
    
    # Define the upsacaled output of the controlled HV prod based on the current index
    upscaled_ctrld_hvProd_max = ctrld_hvProd_max_range[cur_index]  

    # Initialize variables for parallel engines
    dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(df_final,
                                                      networks, 
                                                      cur_hvProd_max = upscaled_ctrld_hvProd_max )

    print(f'***************************       Simulation {cur_index+1}/{len(ctrld_hvProd_max_range)}      ****************')
    print("                           ------- Bloc PF/OPF ---------         ")
    ##          ---------------------------  BLOCK PF/OPF  ------------------------         ##
    # Initialize the parallel engine parameters i.e. send variable into each local space
    opf_status = 'Both'
    df_prodHT_index = dict_df_sgenLoad['df_prodHT'].index # Extract  index of periods.
    
    # Send all the created variables into local spaces of each engine.
    parEngines.sendVar_to_localSpace(df_prodHT_index, 
                                     opf_status, 
                                     dict_df_sgenLoad, 
                                     parameters_dict, 
                                     clean=True )
    # cALL Block pf/op
    ctrld_HvProd_opt = par_block_pfOpf(parEngines, pred_model)    

    # ---------------------- Implement Robust Persistence ------------------------------------
    oriFc.robustness(ctrld_HvProd_opt, 
                     df_final[[ctrld_HvProdName]], 
                     cur_hvProd_max = upscaled_ctrld_hvProd_max )
    
    ##           ---------------------------  BLOCK PROD  ------------------------         ##
    oriFc.block_prod(ctrld_HvProd_opt, 
                     df_final[[ctrld_HvProdName]], 
                     cur_hvProd_max = upscaled_ctrld_hvProd_max )
    #            ------------------------------------------------------------------------  ##
    
    # Create mask
    mask_per2work = per_index2[day_tot_per:]

    # Update Values of the dataframe to use for final power flow bloc with Interest variables 
    # (Real load, real Prod of LV Prod, and Non-COntroled HV Prod, Injected power of Ctrl Prod
    df_final2 =pd.concat([df_final.loc[mask_per2work].iloc[:,[0,1]],
                          df_final.loc[mask_per2work, non_ctrld_hvProdNames_list],
                          ctrld_HvProd_opt.loc[mask_per2work, [ctrld_HvProdName]],
                         ], axis='columns')

    checker.check_networkDataDf_columnsOrder(df_final2)    # Check df_final2 columns order
    
    # Creat dict to send to par Engines based
    dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(df_final2,
                                                      networks )
    ##             ---------------------------  BLOCK PF  ------------------------         ##
    print("                            ------- Bloc PF/OPF ---------         ")    

    opf_status = False
    df_prodHT_index = dict_df_sgenLoad['df_prodHT'].index # Extract  index of periods.
    
    # Send all the created variables into local spaces of each engine.
    parEngines.sendVar_to_localSpace(df_prodHT_index, 
                                     opf_status, 
                                     dict_df_sgenLoad, 
                                     parameters_dict, 
                                     clean=False )
    # Call block pf. It 
    max_vm_pu_pers_df = par_block_pfOpf(parEngines)    

    # TODO : Write a func to save results 
#  #####   Saving Results  --------------------------------------------------------------------------------------
    ctrld_HvProd_df = df_final.loc[df_final2.index, [ctrld_HvProdName]]
    ctrld_HvProd_pers_df = pd.DataFrame(data=ctrld_HvProd_opt.loc[df_final2.index, ctrld_HvProdName].values, 
                                        index=ctrld_HvProd_opt.loc[df_final2.index].index, 
                                        columns=['Pers_Rob Sim'+str(cur_index+1)] )
    # Concat
    ctrld_HvProd_concat_df = pd.concat([ctrld_hvProd_max_range[cur_index]*ctrld_HvProd_df/default_ctrld_hvProd_max,
                                        ctrld_HvProd_pers_df], 
                                       axis='columns')
    # Rename
    ctrld_HvProd_concat_df.rename(columns=({ctrld_HvProdName:'No_Control'}), inplace=True);

    int_sim_res = {'maxV_rise_df':max_vm_pu_pers_df,            # Intermediate results
                   'Power Sgen': ctrld_HvProd_concat_df}         
    sim_res_Dict.update([('Sim'+str(cur_index+1),int_sim_res)]) # Final results

##### Save results of the previous simulation

In [ ]:
joblib.dump(sim_res_Dict,f'{network_folder}simulationResults/SensAnalysisP0100_PersRob.pkl')

-----

#### FUTURE KNOWN

**<span style='color:Orange'>  The results of the Future Known simulation using the default networks (Civaux and St Laurent) is availaible in</span>** [SensAnalysisP0100_FutureKnown](../Pickle_files/simulationResults/SensAnalysisP0100_FutureKnown.pkl). 

In [ ]:
sim_res_Dict = {}  # Dictionnary to save variables

for cur_index in range(len(ctrld_hvProd_max_range)):

    # Define the upsacaled output of the controlled HV prodf
    upscaled_ctrld_hvProd_max = ctrld_hvProd_max_range[cur_index]  
    
    # Initialize variables for parallel engines
    dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(df_final,
                                                      networks, 
                                                      cur_hvProd_max = upscaled_ctrld_hvProd_max )
    
    print(f'***************************       Simulation {cur_index+1}/{len(ctrld_hvProd_max_range)}      ****************')
    print("                           ------- Bloc PF/OPF ---------         ")
    ##          ---------------------------  BLOCK PF/OPF  ------------------------         ##
    # Initialize the parallel engine parameters i.e. send variable into each local space
    opf_status = 'Both'
    df_prodHT_index = dict_df_sgenLoad['df_prodHT'].index # Extract  index of periods.
    
    # Send all the created variables into local spaces of each engine.
    parEngines.sendVar_to_localSpace(df_prodHT_index, 
                                     opf_status, 
                                     dict_df_sgenLoad, 
                                     parameters_dict, 
                                     clean=True )
    # cALL Block pf/op
    ctrld_HvProd_opt = par_block_pfOpf(parEngines)    
    
    # Get maximum voltage over the network 
    max_vm_pu_df_known = ctrld_HvProd_opt[['max_vm_pu']]
    max_vm_pu_df_known.rename(columns={'max_vm_pu':'Fut_Known'}, inplace=True)
    max_vm_pu_pf_df = ctrld_HvProd_opt[['max_vm_pu_pf']] 
    
    
#  #####   Saving Results  --------------------------------------------------------------------------------------
    mask = per_index2[day_tot_per:]
    # Rename simulation result according to the simulation number.
    ctrld_HvProd_known_df = ctrld_HvProd_opt.loc[mask, [ctrld_HvProdName]]
    ctrld_HvProd_known_df.rename(columns={ctrld_HvProdName:'Fut_Known Sim'+str(cur_index+1)}, 
                                 inplace=True)
    # Concat
    ctrld_HvProd_df = df_final.loc[mask, [ctrld_HvProdName]]
    ctrld_HvProd_concat_df = pd.concat([ctrld_hvProd_max_range[cur_index]*ctrld_HvProd_df/default_ctrld_hvProd_max, 
                                        ctrld_HvProd_known_df], 
                                       axis='columns')
    # Rename
    ctrld_HvProd_concat_df.rename(columns=({ctrld_HvProdName:'No_Control'}), inplace=True);


    int_sim_res = {'maxV_rise_df':max_vm_pu_df_known,            # Intermediate results
                   'Power Sgen': ctrld_HvProd_concat_df,
                   'maxV_rise_pf_df':max_vm_pu_pf_df}         
    sim_res_Dict.update([('Sim'+str(cur_index+1),int_sim_res)])   # Final results


##### Save results of the previous simulation

In [ ]:
joblib.dump(sim_res_Dict,f'{network_folder}simulationResults/SensAnalysisP0100_FutureKnown.pkl')

---

#### RNN - Non Robust

**<span style='color:Orange'>  The results of the RNN simulation using the default networks (Civaux and St Laurent) is availaible in</span>** [SensAnalysisP0100_RNN](../Pickle_files/simulationResults/SensAnalysisP0100_RNN.pkl). 

##### Create a function to rename Columns the RNNprediction dataframe name 


    Ex: from ==> ['P0013_RNN', 'P0018_RNN', 'P0100_RNN'] ==> to ['P0013', 'P0018', 'P0100']

In [15]:
rename_cols = lambda name_list : [name.split('_R')[0] for name in name_list]

##### Rename prediction Dataframe

In [16]:
rnnPred_df.rename(columns={cur_name : new_name 
                           for cur_name, new_name in zip(rnnPred_df.columns,rename_cols(rnnPred_df)) }, 
                  inplace=True)

##### Run Simulations

In [ ]:
sim_res_Dict_rnn = {}  # Dictionnary to save variables

for cur_index in range(len(ctrld_hvProd_max_range)): 
    
    # Define the upsacaled output of the controlled HV prod based on the current index
    upscaled_ctrld_hvProd_max = ctrld_hvProd_max_range[cur_index]  

    # Initialize variables for parallel engines
    dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(rnnPred_df,
                                                      networks, 
                                                      cur_hvProd_max = upscaled_ctrld_hvProd_max )

    print(f'***************************       Simulation {cur_index+1}/{len(ctrld_hvProd_max_range)}      ****************')
    print("                           ------- Bloc PF/OPF ---------         ")
    ##          ---------------------------  BLOCK PF/OPF  ------------------------         ##
    # Initialize the parallel engine parameters i.e. send variable into each local space
    opf_status = 'Both'
    df_prodHT_index = dict_df_sgenLoad['df_prodHT'].index # Extract  index of periods.
    
    # Send all the created variables into local spaces of each engine.
    parEngines.sendVar_to_localSpace(df_prodHT_index, 
                                     opf_status, 
                                     dict_df_sgenLoad, 
                                     parameters_dict, 
                                     clean=True )
    # cALL Block pf/op
    ctrld_HvProd_opt = par_block_pfOpf(parEngines)    
    
    ##           ---------------------------  BLOCK PROD  ------------------------         ##
    oriFc.block_prod(ctrld_HvProd_opt, 
                     df_final[[ctrld_HvProdName]], 
                     cur_hvProd_max = upscaled_ctrld_hvProd_max )
    #------------------------------------------------------------------------------
    
    # Create mask
    mask_per2work = per_index2[day_tot_per:]

    # ------  Update Values of the dataframe to use for final power flow bloc with Interest variables 
    # (Real load, real Prod of LV Prod, and Non-COntroled HV Prod, Injected power of Ctrl Prod
    df_final2 =pd.concat([df_final.loc[mask_per2work].iloc[:,[0,1]],
                          df_final.loc[mask_per2work, non_ctrld_hvProdNames_list],
                          ctrld_HvProd_opt.loc[mask_per2work, [ctrld_HvProdName]],
                         ], axis='columns')
    
    checker.check_networkDataDf_columnsOrder(df_final2)    # Check df_final2 columns order
    
    # Creat dict to send to par Engines based
    dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(df_final2,
                                                      networks )
    ##             ---------------------------  BLOCK PF  ------------------------         ##
    print("                            ------- Bloc PF/OPF ---------         ")    
    opf_status = False
    df_prodHT_index = dict_df_sgenLoad['df_prodHT'].index # Extract  index of periods.
    
    # Send all the created variables into local spaces of each engine.
    parEngines.sendVar_to_localSpace(df_prodHT_index, 
                                     opf_status, 
                                     dict_df_sgenLoad, 
                                     parameters_dict, 
                                     clean=False )
    # cALL Block pf
    max_vm_pu_rnn_df = par_block_pfOpf(parEngines) 

#  #####   Saving Results  --------------------------------------------------------------------------------------
    ctrld_HvProd_df = df_final.loc[df_final2.index, [ctrld_HvProdName]]
    
    ctrld_HvProd_rnn_df = ctrld_HvProd_opt.loc[mask, [ctrld_HvProdName]]
    ctrld_HvProd_rnn_df.rename(columns={ctrld_HvProdName:'RNN Sim'+str(cur_index+1)}, 
                                 inplace=True)
    # Concat
    ctrld_HvProd_concat_df = pd.concat([ctrld_hvProd_max_range[cur_index]*ctrld_HvProd_df/default_ctrld_hvProd_max,
                                        ctrld_HvProd_rnn_df],
                                       axis='columns')
    # Rename
    ctrld_HvProd_concat_df.rename(columns=({ctrld_HvProdName:'No_Control'}), inplace=True);

    int_sim_res = {'maxV_rise_df':max_vm_pu_rnn_df,            # Intermediate results
                   'Power Sgen': ctrld_HvProd_concat_df}         
    sim_res_Dict.update([('Sim'+str(cur_index+1),int_sim_res)])# Final results


##### Save results of the previous simulation

In [ ]:
joblib.dump(sim_res_Dict,f'{network_folder}simulationResults/SensAnalysisP0100_RNN.pkl')

#### ROBUST Combined RNN 

**<span style='color:Orange'>  The results of the robust RNN simulation using the default networks (Civaux and St Laurent) are availaible for </span> [P0100_RNN_Robust_Model1](../Pickle_files/simulationResults/SensAnalysisP0100_RNN_Robust_Model1.pkl) <span style='color:orange'> and  </span> [P0100_RNN_Robust_Model3](../Pickle_files/simulationResults/SensAnalysisP0100_RNN_Robust_Model3.pkl)** 

##### Load predicted variables based on both models

In [18]:
pred_num_vRise = joblib.load(f'{network_folder}simulationResults/Numerical_Voltage_Rise_Predicted.pkl')
pred_bin_vRise = joblib.load(f'{network_folder}simulationResults/Binary_Voltage_Rise_Predicted.pkl')

mask_per2work = pred_bin_vRise[pred_bin_vRise.index>='2021 06 02'].index # We select testSet_start_date_M2 + 1 day  of testing set 

##### Define the Model to use for the combined Robust Prediction
The user might choose here, the combination of models to use. We have worked with `"Model1"` and `"Model3"`. See the Documentation of [`oriFunctions.combineRnnPred()`](https://ori-srd.readthedocs.io/en/stable/orifunctions/descriptionOfFunctions.html#combinernnpred) for the possible choices.

In [19]:
paramUser = 'Model3' # Choose model to use for combined Prediction

##### Run Simulations

In [ ]:
sim_res_Dict_rnn_rob = {}  # Dictionnary to save variables

# for cur_index in range(len(ctrld_hvProd_max_range)): 
for cur_index in [10]: 
    
    # Define the upsacaled output of the controlled HV prod based on the current index
    upscaled_ctrld_hvProd_max = ctrld_hvProd_max_range[cur_index]  

    # Initialize variables for parallel engines
    dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(rnnPred_df,
                                                      networks, 
                                                      cur_hvProd_max = upscaled_ctrld_hvProd_max )

    print(f'***************************       Simulation {cur_index+1}/{len(ctrld_hvProd_max_range)}      ****************')
    print("                           ------- Bloc PF/OPF ---------         ")
    ##          ---------------------------  BLOCK PF/OPF  ------------------------         ##
    # Initialize the parallel engine parameters i.e. send variable into each local space
    opf_status = 'Both'
    df_prodHT_index = dict_df_sgenLoad['df_prodHT'].index # Extract  index of periods.
    
    # Send all the created variables into local spaces of each engine.
    parEngines.sendVar_to_localSpace(df_prodHT_index, 
                                     opf_status, 
                                     dict_df_sgenLoad, 
                                     parameters_dict, 
                                     clean=True )
    # cALL Block pf/op
    model1_pred_res = par_block_pfOpf(parEngines)   
    
    ##  Associate each model to its data
    model1_Vrise, model2_Vrise, model3_Vrise = (model1_pred_res.loc[mask_per2work, ['max_vm_pu_pf']],  
                                                pred_bin_vRise.loc[mask_per2work], 
                                                pred_num_vRise.loc[mask_per2work])
    
    # create a dictionnary to save modelVrise
    v_rise_dict = {name: model for name, model in zip(['Model'+str(i) for i in range(1,4)],
                                                      [model1_Vrise, model2_Vrise, model3_Vrise] ) }
    
    # Extract P0100 based on robust prediction of threshold
    ctrld_HvProd_opt, binThresh_df = oriFc.combineRnnPred(v_rise_dict, 
                                                          model1_pred_res[[ctrld_HvProdName]], 
                                                          auth_max_VriseHvBus=vm_mu_max,
                                                          n_models=paramUser)
    # Implement robustness -------------------------------------
    oriFc.robustness(ctrld_HvProd_opt, 
                     df_final[[ctrld_HvProdName]], 
                     cur_hvProd_max = upscaled_ctrld_hvProd_max,
                     combRnn_param = (binThresh_df,paramUser)
                    )
    ##             ---------------------------  BLOCK PROD  ------------------------         ##
    oriFc.block_prod(ctrld_HvProd_opt, 
                     df_final[[ctrld_HvProdName]], 
                     cur_hvProd_max = upscaled_ctrld_hvProd_max )
    #-------------------------------------------------------------------------------------------
    
    # ------  Update Values of the dataframe to use for final power flow bloc with Interest variables 
    # (Real load, real Prod of LV Prod, and Non-COntroled HV Prod, Injected power of Ctrl Prod
    df_final2 =pd.concat([df_final.loc[mask_per2work].iloc[:,[0,1]],
                          df_final.loc[mask_per2work, non_ctrld_hvProdNames_list],
                          ctrld_HvProd_opt.loc[mask_per2work, [ctrld_HvProdName]],
                         ], axis='columns')
    
    # Check df_final2 columns order
    checker.check_networkDataDf_columnsOrder(df_final2)    
    
    # Creat dict to send to par Engines based
    dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(df_final2,
                                                      networks )
    
    #             ---------------------------  BLOCK PF  ------------------------         ##
    print("                            ------- Bloc PF/OPF ---------         ")
    opf_status = False
    df_prodHT_index = dict_df_sgenLoad['df_prodHT'].index # Extract  index of periods.
    
    # Send all the created variables into local spaces of each engine.
    parEngines.sendVar_to_localSpace(df_prodHT_index, 
                                     opf_status, 
                                     dict_df_sgenLoad, 
                                     parameters_dict, 
                                     clean=False )
    # cALL Block pf
    max_vm_pu_rnn_df = par_block_pfOpf(parEngines)  

    #####   Saving Results ----------------------------------------------------------------------
    ctrld_HvProd_df = df_final.loc[df_final2.index, [ctrld_HvProdName]]
    ctrld_HvProd_rnn_df = ctrld_HvProd_opt.loc[mask_per2work, [ctrld_HvProdName]]
    ctrld_HvProd_rnn_df.rename(columns={ctrld_HvProdName:'RNN Sim'+str(cur_index+1)}, 
                               inplace=True)
   # Concat
    ctrld_HvProd_concat_df = pd.concat([ctrld_hvProd_max_range[cur_index]*ctrld_HvProd_df/default_ctrld_hvProd_max,
                                        ctrld_HvProd_rnn_df], 
                                       axis='columns')
    # Rename
    ctrld_HvProd_concat_df.rename(columns=({ctrld_HvProdName:'No_Control'}), inplace=True);


    int_sim_res = {'maxV_rise_df':max_vm_pu_rnn_df,              # Intermediate results
                   'Power Sgen': ctrld_HvProd_concat_df}         
    sim_res_Dict_rnn_rob.update([('Sim'+str(cur_index+1),int_sim_res)])   # Final results


##### Save results of the previous simulation

In [ ]:
joblib.dump(sim_res_Dict_rnn_rob,f'{network_folder}simulationResults/SensAnalysisP0100_RNN_Robust_{paramUser}.pkl')